In [ ]:
!pip uninstall -y transformers sentence-transformers peft
# 안정적인 버전 조합
!pip install transformers==4.40.2
!pip install sentence-transformers==2.2.2
!pip install peft==0.3.0
!pip install torch tqdm scikit-learn --upgrade

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np
from tqdm import tqdm
import os

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# -----------------------------
# 1️⃣ 데이터 로드
# -----------------------------
data_paths = {
    "amazon": "/content/drive/MyDrive/PADA model code/data/new_amazon.csv",
    "audible": "/content/drive/MyDrive/PADA model code/data/new_audible.csv",
    "coursera": "/content/drive/MyDrive/PADA model code/data/new_coursera.csv",
    "hotel": "/content/drive/MyDrive/PADA model code/data/new_hotel.csv"
}

# -----------------------------
# 2️⃣ 임베딩 모델 정의 (RoBERTa → base 버전으로 변경)
# -----------------------------
embedding_models = {
    "T5": "sentence-transformers/sentence-t5-base",
    "Sentence-BERT": "sentence-transformers/all-MiniLM-L6-v2",
    "RoBERTa": "roberta-base",
    "BERT": "bert-base-uncased",
    "DistilBERT": "distilbert-base-uncased"
}

output_dir = "/content/drive/MyDrive/PADA model code/embeddings"
os.makedirs(output_dir, exist_ok=True)

# -----------------------------
# 3️⃣ 일반 Transformer 임베딩 함수 (배치 + 진행률)
# -----------------------------
def get_transformer_embeddings(texts, model_name, tokenizer_name, max_length=256, batch_size=32):
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    model = AutoModel.from_pretrained(model_name).to(device)
    model.eval()

    embeddings = []
    for i in tqdm(range(0, len(texts), batch_size), desc=f"{model_name} Batches"):
        batch_texts = texts[i:i+batch_size]
        inputs = tokenizer(batch_texts, return_tensors="pt", truncation=True,
                           padding="max_length", max_length=max_length).to(device)
        with torch.no_grad():
            outputs = model(**inputs)
            last_hidden = outputs.last_hidden_state
            batch_emb = last_hidden.mean(dim=1).cpu().numpy()  # mean pooling
            embeddings.append(batch_emb)
    return np.vstack(embeddings)

# -----------------------------
# 4️⃣ SentenceTransformer 전용 함수
# -----------------------------
def get_sentence_transformer_embeddings(texts, model_name):
    model = SentenceTransformer(model_name, device=device)
    embeddings = model.encode(texts, batch_size=32, show_progress_bar=True)
    return np.array(embeddings)

# -----------------------------
# 5️⃣ 전체 진행률: 데이터셋 × 모델
# -----------------------------
total_tasks = len(data_paths) * len(embedding_models)
task_counter = 0

for dataset_name, path in data_paths.items():
    df = pd.read_csv(path)
    texts = df['Review_Text'].astype(str).tolist()

    for model_label, model_path in embedding_models.items():
        task_counter += 1
        print(f"\n🌟 [{task_counter}/{total_tasks}] Processing {dataset_name} with {model_label}...")

        if "sentence-transformers" in model_path:
            embs = get_sentence_transformer_embeddings(texts, model_path)
        else:
            embs = get_transformer_embeddings(texts, model_path, model_path)

        # ✅ PCA 완전히 제거 (모델 자체 차원 그대로 저장)
        print(f"Embedding shape: {embs.shape}")

        save_path = os.path.join(output_dir, f"{dataset_name}_{model_label.replace('-', '')}.npy")
        np.save(save_path, embs)
        print(f"✅ Saved: {save_path}")

print("\n🎉 모든 임베딩 파일 저장 완료!")